In [33]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import sys
import bwapiEnums


In [34]:
inputUnitCount = 256
outputUnitCount = 256
output_sequence_length = 5
unitTypeTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.allUnits)
unitActionsTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.unitActions)
unitOrdersTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.unitOrders)

In [35]:
# Input layers
unitTypeInput = layers.Input(shape=(inputUnitCount,), dtype=tf.string, name='UnitTypeInput')
unitOrderInput = layers.Input(shape=(inputUnitCount,), dtype=tf.string, name='UnitOrderInput')
positionInput = layers.Input(shape=(inputUnitCount, 2), dtype=tf.float32, name='PositionInput')

print(unitTypeInput)

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.string, name='UnitTypeInput'), name='UnitTypeInput', description="created by layer 'UnitTypeInput'")


In [36]:
#unitTypeVectorized = layers.Lambda(lambda x: unitTypeTextVectorizationLayer(x))(unitTypeInput)
#unitTypeVectorized = unitTypeTextVectorizationLayer(unitTypeInput)
# Vectorize the unit type and unit order inputs
unitTypeVectorized_list = [unitTypeTextVectorizationLayer(unitTypeInput[:, i]) for i in range(inputUnitCount)]
unitOrderVectorized_list = [unitActionsTextVectorizationLayer(unitOrderInput[:, i]) for i in range(inputUnitCount)]

# Stack the vectorized outputs back to rank-2 tensors
unitTypeVectorized = layers.Concatenate(axis=1)(unitTypeVectorized_list)
unitOrderVectorized = layers.Concatenate(axis=1)(unitOrderVectorized_list)

# Reshape the stacked outputs
unitTypeVectorized = layers.Reshape((inputUnitCount, -1))(unitTypeVectorized)
unitOrderVectorized = layers.Reshape((inputUnitCount, -1))(unitOrderVectorized)

# Convert the vectorized outputs to float32
unitTypeVectorized = tf.cast(unitTypeVectorized, tf.float32)
unitOrderVectorized = tf.cast(unitOrderVectorized, tf.float32)

In [41]:

# Combine the vectorized inputs and position input
combinedInputs = layers.Concatenate(axis=-1)([unitTypeVectorized, unitOrderVectorized, positionInput])
print(combinedInputs)

KerasTensor(type_spec=TensorSpec(shape=(None, 256, 12), dtype=tf.float32, name=None), name='concatenate_12/concat:0', description="created by layer 'concatenate_12'")


In [40]:
# Add more layers as needed for your specific use case
hiddenLayer = layers.Dense(inputUnitCount*10, activation='relu')(combinedInputs)

print(hiddenLayer)
unitOrderOutputLayer = layers.Dense(outputUnitCount * len(bwapiEnums.unitOrders), activation='softmax')(hiddenLayer)
unitOrderPositionOutputLayer = layers.Dense(outputUnitCount*2, activation='relu')(hiddenLayer)
print(unitOrderPositionOutputLayer)
unitOrderPositionOutputLayer = layers.Reshape((outputUnitCount, 2))(unitOrderPositionOutputLayer)

model = keras.models.Model(inputs=[unitTypeInput, unitOrderInput, positionInput], outputs=[unitOrderOutputLayer,unitOrderPositionOutputLayer])


KerasTensor(type_spec=TensorSpec(shape=(None, 256, 2560), dtype=tf.float32, name=None), name='dense_16/Relu:0', description="created by layer 'dense_16'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256, 512), dtype=tf.float32, name=None), name='dense_18/Relu:0', description="created by layer 'dense_18'")


ValueError: Exception encountered when calling layer "reshape_15" (type Reshape).

total size of new array must be unchanged, input_shape = [256, 512], output_shape = [256, 2]

Call arguments received by layer "reshape_15" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 256, 512), dtype=float32)

In [ ]:
model.summary()